# Evaluation pipeline: classifier baselines

In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM
from sentence_transformers import SentenceTransformer
from itertools import chain
import pandas as pd
import numpy as np
import spacy
import operator
from pathlib import Path
from scipy import stats
from gensim.models.wrappers import FastText
from gensim.models import Word2Vec
from tools import animacy_evaluation,processing
import unidecode
from collections import Counter
import sklearn
import pathlib
import pickle
from tools import processing

In [2]:
from tools import classifiers

10/16/2020 10:06:09 - WARNING - gensim.models.deprecated.word2vec -   Slow version of gensim.models.deprecated.word2vec is being used
10/16/2020 10:06:09 - INFO - gensim.models.deprecated.fasttext_wrapper -   loading 2000000 words for fastText model from ../models/language_models/fastai/cc.en.300.bin
10/16/2020 10:08:55 - INFO - gensim.models.deprecated.fasttext_wrapper -   loading weights for 2000000 words for fastText model from ../models/language_models/fastai/cc.en.300.bin
10/16/2020 10:11:39 - INFO - gensim.models.deprecated.fasttext_wrapper -   loaded (2000000, 300) weight matrix for fastText model from ../models/language_models/fastai/cc.en.300.bin


Building sklearn text classifier...


In [3]:
training_corpus = "stories/"
testing_corpus = "stories/"
datascenario = "unique_"
abspath = "/Users/mcollardanuy/Documents/githubCode/AtypicalAnimacy/"

In [4]:
dataset_traindf = pd.read_pickle(abspath + "data/" + training_corpus + datascenario + "train.pkl")
dataset_traindf['both_masked'] = dataset_traindf.apply(lambda row: processing.determine_context("maskedSentence", row, "both"), axis=1)
dataset_traindf['both_unmasked'] = dataset_traindf.apply(lambda row: processing.determine_context("currentSentence", row, "both"), axis=1)

dataset_testdf = pd.read_pickle(abspath + "data/" + testing_corpus + datascenario + "train.pkl")
dataset_testdf['both_masked'] = dataset_testdf.apply(lambda row: processing.determine_context("maskedSentence", row, "both"), axis=1)
dataset_testdf['both_unmasked'] = dataset_testdf.apply(lambda row: processing.determine_context("currentSentence", row, "both"), axis=1)

In [7]:
for prp_scenario in ["_with_prp", "_without_prp"]:
    if prp_scenario == "_without_prp":
        dataset_traindf = dataset_traindf[dataset_traindf["targetIsPRP"] == False]
        dataset_testdf = dataset_testdf[dataset_testdf["targetIsPRP"] == False]
    
    for classifier in ["tfidf_svm", "wemb_svm"]:
        for type_of_training_data in ["targetExpression", "context3wmasked", "context3w", "both_masked", "both_unmasked", "currentSentence", "maskedSentence"]:
        
            threshold_list = list(np.arange(0, 1.05, 0.1))

            df_results = pd.DataFrame(columns = ['threshold', 'precision', 'recall', 'fscore', 'micro_fscore', 'map'])
            print(prp_scenario, type_of_training_data, classifier)

            if not Path(abspath + "experiments/" + training_corpus + "classifier_" + datascenario + "train_" + classifier + "_" + type_of_training_data + prp_scenario + ".tsv").exists():

                # -------------------------------------------------
                # Optimal threshold from train:
                for th in threshold_list:
                    print("* Tuning parameters.")
                    th = float(round(th,2))
                    y_pred = classifiers.classify(abspath,classifier,dataset_traindf[type_of_training_data],type_of_training_data,training_corpus,datascenario,prp_scenario)
                    y_true = [x for x in dataset_traindf['animated'].tolist()]

                    precision, recall, fscore, micro_fscore,map_ = animacy_evaluation.results(y_true,y_pred,th)
                    df_results = df_results.append({'threshold':th, 'precision':round(precision,3), 'recall':round(recall,3), 'fscore':round(fscore,3), 'micro_fscore':round(micro_fscore,3), 'map':round(map_,3)}, ignore_index=True)

                df_results.sort_values(by='fscore', ascending=False).to_csv(abspath + "experiments/" + training_corpus + "classifier_" + datascenario + "train_" + classifier + "_" + type_of_training_data + prp_scenario + ".tsv", sep="\t")

            # -------------------------------------------------
            # Apply to test:
            parameters_best = pd.read_csv(abspath + "experiments/" + training_corpus + "classifier_" + datascenario + "train_" + classifier + "_" + type_of_training_data + prp_scenario + ".tsv", sep="\t").iloc[0]
            print(abspath + "experiments/" + training_corpus + "classifier_" + datascenario + "train_" + classifier + "_" + type_of_training_data + prp_scenario + ".tsv")
            obs_threshold = parameters_best['threshold']
            print(obs_threshold, dataset_testdf.shape)

            y_pred = classifiers.classify(abspath,classifier,dataset_traindf[type_of_training_data],type_of_training_data,training_corpus,datascenario,prp_scenario)
            y_true = [x for x in dataset_testdf['animated'].tolist()]

            precision, recall, fscore, micro_fscore,map_ = animacy_evaluation.results(y_true,y_pred,obs_threshold)

            print(type_of_training_data, classifier, "(t=" + str(round(obs_threshold,2)) + ") & " + str(round(precision,3)) + " & " + str(round(recall,3)) + " & " + str(round(fscore,3)) + " & " + str(round(map_,3)))

_with_prp targetExpression tfidf_svm
/Users/mcollardanuy/Documents/githubCode/AtypicalAnimacy/experiments/stories/classifier_unique_train_tfidf_svm_targetExpression_with_prp.tsv
0.3 (8324, 12)
targetExpression tfidf_svm (t=0.3) & 0.919 & 0.87 & 0.894 & 0.922
_with_prp context3wmasked tfidf_svm
/Users/mcollardanuy/Documents/githubCode/AtypicalAnimacy/experiments/stories/classifier_unique_train_tfidf_svm_context3wmasked_with_prp.tsv
0.7 (8324, 12)
context3wmasked tfidf_svm (t=0.7) & 0.756 & 0.781 & 0.768 & 0.896
_with_prp context3w tfidf_svm
/Users/mcollardanuy/Documents/githubCode/AtypicalAnimacy/experiments/stories/classifier_unique_train_tfidf_svm_context3w_with_prp.tsv
0.6 (8324, 12)
context3w tfidf_svm (t=0.6) & 0.825 & 0.803 & 0.814 & 0.914
_with_prp both_masked tfidf_svm
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* T

currentSentence tfidf_svm (t=0.4) & 0.812 & 0.815 & 0.813 & 0.849
_without_prp maskedSentence tfidf_svm
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
/Users/mcollardanuy/Documents/githubCode/AtypicalAnimacy/experiments/stories/classifier_unique_train_tfidf_svm_maskedSentence_without_prp.tsv
0.4 (3899, 12)
maskedSentence tfidf_svm (t=0.4) & 0.794 & 0.8 & 0.797 & 0.834
_without_prp targetExpression wemb_svm
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
* Tuning parameters.
/Users/mcollardanuy/Documents/githubCode/AtypicalAnimacy/experiments/stories/classifier_unique_train_wemb_svm_targetExpression_without_prp.tsv
0.4 (3899, 12)
targetExpression 